In [1]:
# 3. control sample rate;
# 4. mAP

# 卡在baseline这里了，因为输出的mask 很奇怪，全0；


In [2]:
# # epoch acc

# def valid_look(fold_id, pth=None):
#     # 由于 img_ori shape 不一致，所以无法放到 loader 里以 batch的形式呈现，因此，bs必须等于1；
#     # 之所以要 img_ori, 是因为想要画图展示，想要计算ori_bbox

#     if pth is None:
#         param = torch.load('./models/unet_49' + '.pth')  # stage3 use model pretrained with pseudo-labels
#         model.load_state_dict(param)  # initialize with pretained weight
#     model = model.to(device)
#     model.eval()
#     valid_data_tmp = Jiu_valid(fold_id=fold_id, mode='valid')
#     loader = DataLoader(valid_data_tmp,
#                             batch_size=1,
#                             num_workers=8,
#                             pin_memory=True)
#     i = 0
#     c_pred, c_label_list, bb1_list, bb1_list, img_w, img_h = [],[],[],[],[],[]
#     for imgs, mask, C, bbox, H, W, img_ori in progress_bar(loader):   
#         i+=1
#         if i == 1:
#             imgs = imgs.to(device)
#             with torch.no_grad():
#                 mask_pred = model(imgs).to('cpu').softmax(1).argmax(1)[0] # w, h
#                 bbox_pred = mask2bbox_withscale(mask_pred.numpy(), H, W)
#                 img = img_ori[0].numpy() # h,w,3
#                 x, y, w, h = bbox.numpy()[0]
#                 x1, y1, w1, h1 = [i.tolist()[0] for i in bbox_pred]
#                 c_label_list.append(C)
#                 bb1_list.append([x, y, w, h])
#                 bb2_list.append([x1, y1, w1, h1])
#                 img_w.append(W)
#                 img_h.append(H)
#                 if 1:
#                     cv2.rectangle(img, (x, y), (x + w, y + h), (1, 0, 0), 2)
#                     cv2.rectangle(img, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 1), 2)
#                     vis.image(img_gt.transpose((2,0,1)), win='valid1', opts={'title':'valid_gt_{}'.format(C)})
#                     vis.image(img_pred.transpose((2,0,1)), win='valid1', opts={'title':'img_pred'})
#         else:
#             break
#         weight_dic = {1:0.15,
#                     2:0.09,
#                     3:0.09,
#                     4:0.05,
#                     5:0.13,
#                     6:0.05,
#                     7:0.12,
#                     8:0.13,
#                     9:0.07,
#                     10:0.12}
#     mAp = mAP(weight_dic, c_pred, c_label_list, bb1_list, bb1_list, img_w, img_h)
#     return mAp



In [3]:
import cv2
import os
import torch
import json
from copy import deepcopy
import ipdb
import numpy as np
from torchvision import models
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler
import matplotlib.pyplot as plt
from data import JiuData, do_resize, JiuTest
from model import *
from augment import *
from sampler import JiuSampler
from loss import multi_class_entropy, lovasz_softmax, FocalLoss2d, DiceLoss2D, FocalLoss2D_2
from fastprogress.fastprogress import master_bar, progress_bar
from eval import mAP, mask2bbox_withscale, mask2bbox
from timeit import default_timer as timer
from visdom import Visdom
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

def time_to_str(t, mode='min'):
    from timeit import default_timer as timer
    
    if mode=='min':
        t  = int(t)/60
        hr = t//60
        min = t%60
        return '%2d hr %02d min'%(hr,min)
    elif mode=='sec':
        t   = int(t)
        min = t//60
        sec = t%60
        return '%2d min %02d sec'%(min,sec)
    else:
        raise NotImplementedError


def train(loader, model, dataset, device):
    model = model.to(device)
    running_loss = 0.
    model.train()
    for imgs, masks, cls, bbox, H, W in progress_bar(loader, parent=mb):
        optimizer.zero_grad()
#         ipdb.set_trace()
        imgs, masks, cls = imgs.to(device), masks.to(device), cls.to(device)
        with torch.set_grad_enabled(True):
            logits, cls_logits = model(imgs) # [bs, cls, H, W] [bs, cls]
#             loss1 = multi_class_entropy(logits, masks.squeeze().int())
            loss1 = FocalLoss2D_2(num_class=11, device=device)(logits, masks.squeeze().int())
            loss2 = DiceLoss2D(cls_num=11, device=device)(logits, masks.squeeze().int())
#             loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
#             loss3 = torch.nn.CrossEntropyLoss()(cls_logits, cls)
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
        running_loss += loss.item()*imgs.size(0)
    return running_loss/len(dataset)


def valid(loader, model, dataset, device):
    model = model.to(device)
    run_loss, run_loss1, run_loss2, run_loss3 = 0, 0, 0, 0
    model.eval()
    for imgs, masks, cls, bbox, H, W in progress_bar(loader, parent=mb):    
        imgs, masks, cls = imgs.to(device), masks.to(device), cls.to(device)
        with torch.no_grad():
            logits, cls_logits = model(imgs)
#             loss1 = multi_class_entropy(logits, masks.squeeze().int())
            loss1 = FocalLoss2D_2(num_class=11, device=device)(logits, masks.squeeze().int())
#             loss2 = lovasz_softmax(logits.squeeze(), masks.squeeze().int(), per_image=False)
            loss2 = DiceLoss2D(cls_num=11, device=device)(logits, masks.squeeze().int())
#             loss3 = torch.nn.CrossEntropyLoss()(cls_logits, cls)
            loss = loss1 + loss2
        run_loss += loss.item()*imgs.size(0)
        run_loss1 += loss1.item()*imgs.size(0)
        run_loss2 += loss2.item()*imgs.size(0)
        run_loss3 += 0
    return run_loss/len(dataset), run_loss1/len(dataset), run_loss2/len(dataset), run_loss3/len(dataset)


def valid_look(model, fold_id, epoch, show_num=3, pth=1):
    # logit # bs, 11, h, w 
    # mask_pred
    # bbox_pred
    # picture
    if pth is None:
        param = torch.load('./models/unet_49' + '.pth')  # stage3 use model pretrained with pseudo-labels
        model.load_state_dict(param)  # initialize with pretained weight
    model = model.to(device)
    model.eval()
    valid_data_tmp = JiuData(fold_id=fold_id, mode='valid', return_ori_img=True)
    loader = DataLoader(valid_data_tmp,
#                         sampler = RandomSampler(valid_data_tmp),
                        batch_size=1,
                        num_workers=8,
                        pin_memory=True)
    i = 0
    for imgs, mask, C, bbox, H, W, img_ori in loader:   
        if i < show_num:
            imgs = imgs.to(device)
            with torch.no_grad():
                mask_pred, cls_logit = model(imgs) # [num_cls, w, h]
                mask_pred = mask_pred.to('cpu').softmax(1).argmax(1)[0]
                bbox_pred = mask2bbox_withscale(mask_pred.numpy(), H, W)
                img  = img_ori[0].numpy() # h,w,3
                img_2 = deepcopy(img)
                x, y, w, h = bbox.numpy()[0]
                x1, y1, w1, h1 = [i.tolist()[0] for i in bbox_pred]
                print('mask pred rectangle: {}/{}'.format((x1, y1), (x1 + w1, y1 + h1)))
                cv2.rectangle(img, (x, y), (x + w, y + h), (1, 0, 0), 2)
                cv2.rectangle(img_2, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 1), 2)
#                 ipdb.set_trace()
                import matplotlib.pyplot as plt
                tmp = np.array([0,1,2])+i*3
                plt.subplot(331 + tmp[0])
                plt.imshow(mask_pred)
                plt.title('mask_epoch:{}'.format(epoch))
                
                plt.subplot(331 + tmp[1])
                plt.imshow(img)
                plt.title('val_gt')
                
                plt.subplot(331 + tmp[2])
                plt.imshow(img_2)
                plt.title('val_pred_{}'.format(valid_data_tmp.id2cls[C.tolist()[0]]))
                
                i+=1
            vis.matplot(plt) 
        else:
            break
    return 


def infer_upload(pre_train_pth=None):
    if pre_train_pth is None:
        param = torch.load('./models/unet_49' + '.pth')  # stage3 use model pretrained with pseudo-labels
        model.load_state_dict(param)  # initialize with pretained weight
    model = model.to(device)
    model.eval()
    dic = {}
    dic['images'] = []
    dic['annotations']=[]
    for index, tup in enumerate(progress_bar(test_loader)):  
        name, imgs, ori_h, ori_w = tup
        dic['images'].append({'file_name': name,
                              'id': index+1})

        imgs = imgs.to(device)
        with torch.no_grad():
            mask = model(imgs).to('cpu').softmax(1).argmax(1)[0] #w,h
            bbox_pred = mask2bbox_withscale(mask.numpy(), ori_h, ori_w)
            dic['annotations'].append({'image_id':index+1,
                           'bbox':bbox_pred,
                           'category_id':mask.max(),
                           'score':1})
    return dic


weight_dic = {1:0.15,
2:0.09,
3:0.09,
4:0.05,
5:0.13,
6:0.05,
7:0.12,
8:0.13,
9:0.07,
10:0.12}

In [4]:
batch_size = 20
# model = Unet()
model = Unet_qb(num_class=11, HyperColumn=True)
lr = 0.01
min_lr = 0.00015
EPOCH=200
snapshot = 2
scheduler_step = EPOCH//snapshot
device = 'cuda'
fold_id = 1
max_batchs_per_epoch = 300

train_data = JiuData(fold_id=fold_id, mode='train', return_ori_img=False)
train_loader = DataLoader(
                    train_data,
#                     sampler=JiuSampler(train_data, bs=batch_size, max_batchs_per_epoch=max_batchs_per_epoch),
                    batch_size=batch_size,
                    num_workers=8,
                    pin_memory=True)

valid_data = JiuData(fold_id=fold_id, mode='valid', return_ori_img=False)
valid_loader = DataLoader(
                    valid_data,
                    shuffle=RandomSampler(valid_data),
                    batch_size=batch_size,
                    num_workers=8,
                    pin_memory=True)

testset = JiuTest()
test_loader = DataLoader(
                    testset,
                    batch_size=1,
                    num_workers=8,
                    pin_memory=True)


In [5]:
# !python -m visdom.server

In [6]:
vis = Visdom(env='jiujiu4')

Setting up a new session...


In [ ]:
optimizer = torch.optim.SGD(params = model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_step, eta_min=min_lr)
start = timer()
mb = master_bar(range(EPOCH))

for epoch in mb:
    
#     # DEBUG
#     if 1:
#         PATH='./models/unet_aug01_'+ str(49) + '.pth'
#         model.load_state_dict(torch.load(PATH))
#         valid_look(model, fold_id)
    
    # training;
    scheduler.step()
    epoch_loss = train(train_loader, model, train_data, device)
    l_all, l1, l2, l3 = valid(valid_loader, model, valid_data, device)
    valid_look(model, fold_id, epoch)
    vis.line(X=[epoch], Y=[[scheduler.get_lr()[0], epoch_loss, l_all, l1, l2, l3]], opts=dict(markers=True, showlegend=True), win='loss', update='append' if epoch>0 else None)
    

    # 重置lr;
    if (epoch + 1) % scheduler_step == 0:
        torch.save(model.state_dict(),  './models/unet_qb_aug_focal'+ str(epoch) + '.pth')
        optimizer = torch.optim.SGD(params = model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=scheduler_step, eta_min=min_lr)
        scheduler.step()
        

mask pred rectangle: (0, 0)/(0, 0)
mask pred rectangle: (0, 0)/(0, 0)
mask pred rectangle: (0, 0)/(0, 0)
